- docs: https://face-recognition.readthedocs.io/en/latest/face_recognition.html
- face recognition used: https://github.com/ageitgey/face_recognition

## For testing only

In [1]:
import cv2
import face_recognition
""" print(cv2.__version__) """
# yall gottta add ur own images and change the path to the image if u wanna test too.
img = cv2.imread('faces/ryan_reynolds.jpg')
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encodings = face_recognition.face_encodings(rgb_img)[0]

img2 = cv2.imread('faces/ryan_reynolds1.jpg')
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img_encodings2 = face_recognition.face_encodings(rgb_img2)[0]

img3 = cv2.imread('faces/ryan_gosling.webp')
rgb_img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img_encodings3 = face_recognition.face_encodings(rgb_img3)[0]

img4 = cv2.imread('faces/ryan_gosling1.webp')
rgb_img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img_encodings4 = face_recognition.face_encodings(rgb_img4)[0]

""" cv2.imshow('Ryan Reynolds', img)
cv2.imshow('Ryan Reynolds 1', img2)
cv2.imshow('Ryan Gosling', img3)
cv2.waitKey(0) """

print(img_encodings)

[-1.82547867e-01  2.60341614e-01 -3.46864201e-02 -9.15299580e-02
 -1.49315372e-01  1.85766201e-02 -8.29656571e-02 -4.74582799e-02
  1.52570933e-01 -4.10558097e-03  2.31237501e-01 -5.53767011e-02
 -2.46654317e-01 -1.63636487e-02 -3.20855975e-02  8.40576440e-02
 -2.21513018e-01 -4.06846851e-02 -1.00537993e-01  2.24268287e-02
  9.92056280e-02  1.12263098e-01  6.33522496e-02  8.33507329e-02
 -6.60814941e-02 -3.64214569e-01 -3.76806669e-02 -1.21768683e-01
  1.38467580e-01 -1.14823304e-01  7.51571655e-02 -1.53866503e-02
 -1.46745682e-01 -4.24677879e-02 -2.95799226e-04  3.27303186e-02
 -4.91484031e-02 -2.27554478e-02  2.29331866e-01  1.24631012e-02
 -1.29686415e-01  8.14248621e-02  3.16107459e-02  3.33386540e-01
  1.74130887e-01  4.96648028e-02  1.63773596e-02 -1.08795717e-01
  6.52691349e-02 -3.02975237e-01  1.23957567e-01  2.17035338e-01
  1.29369780e-01  6.82895258e-02  1.27793059e-01 -1.33649930e-01
  4.69275862e-02  1.96162939e-01 -2.55120754e-01  1.52502477e-01
  1.14901781e-01  3.17177

In [3]:
result = face_recognition.compare_faces([img_encodings], img_encodings2) # Should return True
print(result)

[np.True_]


In [4]:
result = face_recognition.compare_faces([img_encodings3], img_encodings4) # Should return True
print(result)

[np.True_]


In [5]:
result = face_recognition.compare_faces([img_encodings], img_encodings3) # Should return False
print(result)

[np.False_]


In [6]:
result = face_recognition.compare_faces([img_encodings2], img_encodings4) # Should return False
print(result)

[np.False_]


## Using real time video feed using webcam
- create a folder called faces
- add your image to the faces folder and pictures of other people to test

In [2]:
import cv2
import face_recognition
import numpy as np
import math
import os, sys

# Define face_confidence function
def face_confidence(face_distance, face_match_threshold=0.6):
    if face_distance > face_match_threshold:
        range = (1.0 - face_match_threshold)
        linear_val = (1.0 - face_distance) / (range * 2.0)
        return round(linear_val,2) #rounding to 2 decimal places
    else:
        range = face_match_threshold
        linear_val = 1.0 - (face_distance / (range * 2.0))
        return round(linear_val,2)
    
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True #Reccommended from docs saves processing power not making it detect faces for every frame

    def __init__(self):
        self.encode_faces()

    def encode_faces(self):
        for image in os.listdir('faces'):
            face_image = face_recognition.load_image_file(f'faces/{image}')
            face_enc = face_recognition.face_encodings(face_image)[0]

            self.known_face_encodings.append(face_enc)
            self.known_face_names.append(image)
        print(self.known_face_names)

    def video_recognition(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error: Could not open video.')
            sys.exit()

        while True:
            ret, frame = cap.read()

            if self.process_current_frame:
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # Find face in frame
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                #recognition part
                self.face_names = []
                for face_encoding in self.face_encodings:
                    """ match = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=0.5) """ ## tolerance is the threshold for face matching. This is just for testing
                    match = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    # default name and confidence
                    name = '???'
                    confidence = '???'

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if match[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])
                    
                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame #only process every other frame

            # Drawing the binding box
            for(top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top*=4
                right*=4
                bottom*=4
                left*=4

                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                """ cv2.rectangle(frame, (left, bottom-35), (right, bottom), (255, 0, 0), -1) # """
                cv2.putText(frame, name, (left+6, bottom-6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
            
            cv2.imshow('Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



if __name__ == '__main__':
    fr = FaceRecognition()
    fr.video_recognition()

['me.JPG', 'ryan_gosling.webp', 'ryan_gosling1.webp', 'ryan_reynolds.jpg', 'ryan_reynolds1.jpg']


## DLIB

## Functions to generating, saving and loading embedded face data for model

In [ ]:
import dlib
print(dlib.__version__) 

In [1]:
import cv2
import dlib
import numpy as np
import pickle
import os

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Function to compute the 128D face embedding
def get_face_embedding(face_image, shape):
    face_embedding = face_rec_model.compute_face_descriptor(face_image, shape)
    return np.array(face_embedding)

# Function to load an image, detect a face, and compute the embedding
def get_embedding_from_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = detector(gray)
    
    if len(faces) == 0:
        print("No face found in the image.")
        return None, None
    
    # Get the shape of the first detected face (assuming one face per image)
    face = faces[0]
    shape = predictor(gray, face)
    
    # Get the embedding for the face
    embedding = get_face_embedding(image, shape)
    
    return embedding, image

# Function to compare two face embeddings using Euclidean distance
def compare_faces(known_embedding, new_embedding, threshold=0.6):
    distance = np.linalg.norm(known_embedding - new_embedding)
    return distance < threshold  # Return True if the distance is below the threshold

# Initialize known face embeddings and names from the pickle file, if it exists
def load_embeddings_from_pickle(pickle_file):
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as f:
            return pickle.load(f)
    else:
        return [], []  # Return empty lists if pickle file doesn't exist

# Function to save embeddings and names to a pickle file
def save_embeddings_to_pickle(pickle_file, embeddings, names):
    with open(pickle_file, 'wb') as f:
        pickle.dump((embeddings, names), f)
    print(f"Saved embeddings and names to {pickle_file}")

# Load existing embeddings and names
pickle_file = "face_embeddings.pkl"
known_face_embeddings, known_face_names = load_embeddings_from_pickle(pickle_file)

# Function to add a new person by uploading an image and getting their embedding
def add_new_face(image_path, name):
    global known_face_embeddings, known_face_names
    embedding, _ = get_embedding_from_image(image_path)
    if embedding is not None:
        known_face_embeddings.append(embedding)
        known_face_names.append(name)
        save_embeddings_to_pickle(pickle_file, known_face_embeddings, known_face_names)
        print(f"Added {name} to the known faces database.")




In [6]:
# To add image
image_path = "faces/me.jpg" 
name = "Darren"
add_new_face(image_path, name)

RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [13]:
pickle_file = "face_embeddings.pkl"

# Load the contents of the pickle file
with open(pickle_file, 'rb') as f:
    data = pickle.load(f)
    """ print(data) """
    # Print the contents
    known_face_embeddings, known_face_names = data
    print("Known Face Names:", known_face_names)
    print("Known Face Embeddings:")
    for embedding in known_face_embeddings:
        print(embedding)

Known Face Names: ['Darren', 'Ryan Gosling']
Known Face Embeddings:
[-0.01868058  0.08564011  0.05817194 -0.02646313 -0.02929644 -0.01893535
 -0.1487243  -0.11367936  0.13209692 -0.09891735  0.19960441  0.00720049
 -0.20630121 -0.10474695 -0.05116588  0.17420474 -0.16647254 -0.08982423
 -0.00370136 -0.00199144  0.05576181  0.04511987  0.06549055 -0.00434251
 -0.03118888 -0.35392129 -0.10605627 -0.11295271  0.00278116 -0.00979992
 -0.10427234  0.08100738 -0.07713564 -0.03595746  0.07537783  0.13721099
  0.00092566  0.01421921  0.21423632  0.0046144  -0.19808899  0.01770782
  0.03399879  0.23606861  0.2442279   0.0570185   0.05939772 -0.12357722
  0.15866596 -0.17130978  0.08902176  0.18414944  0.13823576  0.09966705
  0.03433473 -0.13177553  0.04618246  0.06724964 -0.06711772  0.00300048
  0.12534013 -0.08083674  0.0229241   0.00899508  0.26514316  0.03822265
 -0.14637649 -0.17102857  0.09101833 -0.12432031 -0.13268389  0.02721943
 -0.22579749 -0.11627074 -0.32974693  0.0160929   0.4004

## Function for running real time face detection

In [15]:
# Function to run face recognition on a live webcam feed
def run_face_recognition():
    min_face_size = 350 # Minimum bounding box size to detect a face. Used to filter out people who are far away
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    while True:
        # Capture frame-by-frame from the webcam
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = detector(gray, 1)
        for face in faces:
            x, y, w, h = face.left(), face.top(), face.right(), face.bottom()
            print(f"Face found at X: {x}, Y: {y}, W: {w}, H: {h}")
            if w >= min_face_size and h >= min_face_size:
                """ start_time = time.time() """
                shape = predictor(gray, face)# Get the landmarks/parts for the face
                new_face_embedding = get_face_embedding(frame, shape)# Compute the face embedding for the detected face
            
                cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
                # Compare the new face embedding with known faces
                recognized_name = "Unknown"
                for i, known_embedding in enumerate(known_face_embeddings):
                    if compare_faces(known_embedding, new_face_embedding):
                        recognized_name = known_face_names[i]
                        break
                """ print(f"Recognised {recognized_name} in {time.time() - start_time:.8f} seconds") """

                # Bounding box
                cv2.putText(frame, recognized_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Display the resulting frame with face detections and names
        cv2.imshow('Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture when done
    cap.release()
    cv2.destroyAllWindows()

In [16]:
run_face_recognition()

Face found at X: 390, Y: 217, W: 613, H: 440
Face found at X: 390, Y: 217, W: 613, H: 440
Face found at X: 390, Y: 217, W: 613, H: 440
Face found at X: 349, Y: 201, W: 617, H: 468
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 291, Y: 217, W: 513, H: 440
Face found at X: 315, Y: 192, W: 538, H: 415
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 192, W: 588, H: 415
Face found at X: 340, Y: 217, W: 563, H: 440
Face found at X: 319, Y: 201, W: 587, H: 468
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found at X: 365, Y: 217, W: 588, H: 440
Face found

## Testing for saving the face embeddings for future uses in database

In [6]:
import pickle

with open('face_embeddings.pkl', 'wb') as f:
    pickle.dump((known_face_embeddings, known_face_names), f)

print("Face embeddings and names saved successfully.")



Face embeddings and names saved successfully.


In [7]:
import pickle

# Load the embeddings and names from the .pkl file
with open('face_embeddings.pkl', 'rb') as f:
    known_face_embeddings, known_face_names = pickle.load(f)

print("Face embeddings and names loaded successfully.")
print(known_face_names) #for checking


Face embeddings and names loaded successfully.
['Me']
